In [4]:
import redis
from dotenv import load_dotenv
import os

load_dotenv()

redis_url = os.getenv("REDIS_URL")

In [5]:
def send_maintenance_command(process_name):
    try:
        # Redis 연결 설정
        redis_client = redis.from_url(
            url=redis_url,
            decode_responses=True
        )

        # 점검 명령 발행
        channel = f"{process_name}_maintenance"
        message = "maintenance_request"
        redis_client.publish(channel, message)
        print(f"Sent maintenance command to {channel}: {message}")

    except Exception as e:
        print(f"Failed to send maintenance command: {e}")

In [53]:
send_maintenance_command("P1-A")

Sent maintenance command to P1-A_maintenance: maintenance_request


In [1]:
from dotenv import load_dotenv
import os
from influxdb_client import InfluxDBClient, QueryApi
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [2]:
load_dotenv()

influxdb_url = os.getenv("INFLUXDB_URL")
influxdb_token = os.getenv("INFLUXDB_TOKEN")
influxdb_org = os.getenv("INFLUXDB_ORG")

client = InfluxDBClient(
    url=influxdb_url,
    token=influxdb_token,
    org=influxdb_org
)
query_api = client.query_api()

In [3]:
def query_influxdb(bucket, query):
    tables = query_api.query(f'from(bucket:"{bucket}") |> {query}')
    results = []
    for table in tables:
        for record in table.records:
            results.append({
                "time": record.get_time().strftime('%Y-%m-%d %H:%M:%S'),
                "measurement": record.get_measurement(),
                "fields": record.values
            })
    return results

In [4]:
llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
)

class AgentState:
    query: str
    influx_results: list
    state_graph: StateGraph

def influx_tool_node(state):
    bucket = state.get('bucket', 'process')
    influx_query = state['query']
    results = query_influxdb(bucket, influx_query)
    return {'influx_results': results}

In [ ]:
from langgraph.prebuilt import ToolNode

influx_node = ToolNode(influx_tool_node)

# 요약 노드
def summary_node(state):
    influx_results = state['influx_results']

    prompt = PromptTemplate.from_template("""
    당신은 제조 공정 전문가입니다. 다음 데이터를 요약하여 생산 상황 및 문제점을 짧고 명확하게 정리해 주세요.
    
    데이터:
    {results}
    
    요약:
    """)

    response = llm.invoke(prompt.format(results=influx_results))
    return {"summary": response.content}

# LangGraph 그래프 구성
graph = StateGraph(AgentState)
graph.add_node("influx_query", influx_node)
graph.add_node("summarize", summary_node)

graph.set_entry_point("influx_query")
graph.add_edge("influx_query", "summarize")
graph.add_edge("summarize", END)

agent_app = graph.compile()

ValueError: Function must have a docstring if description not provided.